
<a href="https://colab.research.google.com/github/takzen/financial-ai-engineering-showcase/blob/main/notebooks/week_01_fundamentals/06_project_pdf_analyst.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 📄 Tydzień 1, Dzień 6: Projekt - PDF Financial Analyst (Gemini Edition)

To finałowy projekt pierwszego tygodnia. Zbudujemy pipeline AI typu "End-to-End".
Nasz program będzie czytał dokumenty (PDF) i pisał z nich raporty inwestycyjne.

**Cele na dziś:**
1.  **PDF Parsing:** Wyciąganie tekstu z dokumentów (`pypdf`).
2.  **Google Gemini API:** Łączenie się z modelem `gemini-2.5-flash`.
3.  **Prompt Engineering:** Nadawanie modelowi roli "Senior Analyst".
4.  **Generowanie Danych:** Stworzenie fikcyjnego raportu do testów.

---
### 🛠️ 1. Instalacja

Potrzebujemy:
*   `langchain-google-genai`: Obsługa modelu Gemini.
*   `pypdf`: Czytanie PDF.
*   `reportlab`: Generowanie PDF (dane testowe).

In [ ]:
!uv add langchain-google-genai pypdf reportlab python-dotenv

### 📝 2. Generowanie Danych (Fikcyjny Raport)

Stworzymy własny plik PDF z wynikami fikcyjnej spółki **"FutureCorp Inc."**.
Zapiszemy go w folderze `data/reports`.

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
import os

# Ścieżki: notebooks/week_01 -> notebooks -> project -> data -> reports
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "../../"))
reports_dir = os.path.join(project_root, "data", "reports")
os.makedirs(reports_dir, exist_ok=True)

pdf_path = os.path.join(reports_dir, "financial_report_q3.pdf")

def create_dummy_pdf(filename):
    c = canvas.Canvas(filename, pagesize=A4)
    
    # Nagłówek
    c.setFont("Helvetica-Bold", 20)
    c.drawString(50, 800, "FutureCorp Inc. - Q3 2024 Financial Report")
    
    # Treść
    c.setFont("Helvetica", 12)
    text_lines = [
        "Date: October 25, 2024",
        "Sector: Technology / AI Hardware",
        "",
        "Executive Summary:",
        "FutureCorp delivered strong results this quarter, driven by unprecedented demand",
        "for our generative AI chips. Revenue grew by 45% YoY to $12.5 billion.",
        "However, operating expenses increased significantly due to R&D investments.",
        "",
        "Key Metrics:",
        "- Revenue: $12.5B (up 45% YoY)",
        "- Net Income: $3.2B (up 12% YoY)",
        "- EPS: $2.45",
        "",
        "Risks & Challenges:",
        "Supply chain constraints remain a bottleneck. Geopolitical tensions in Asia",
        "could impact our semiconductor fabrication partners.",
        "Additionally, a new lawsuit from a competitor regarding patent infringement",
        "could result in fines up to $500M.",
        "",
        "Outlook:",
        "We expect Q4 revenue to be flat due to seasonal adjustments."
    ]
    
    y = 750
    for line in text_lines:
        c.drawString(50, y, line)
        y -= 20
        
    c.save()
    print(f"✅ Wygenerowano raport: {filename}")

create_dummy_pdf(pdf_path)

### 📖 3. PDF Parsing (Ekstrakcja Tekstu)

LLM widzi tekst, nie pliki. Użyjemy `pypdf`, aby zamienić plik binarny na stringa.

In [9]:
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    """Czyta tekst z pliku PDF strona po stronie."""
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {e}"

# Testujemy
raw_text = extract_text_from_pdf(pdf_path)

print("--- TREŚĆ PLIKU PDF ---")
print(raw_text)

--- TREŚĆ PLIKU PDF ---
FutureCorp Inc. - Q3 2024 Financial Report
Date: October 25, 2024
Sector: Technology / AI Hardware
Executive Summary:
FutureCorp delivered strong results this quarter, driven by unprecedented demand
for our generative AI chips. Revenue grew by 45% YoY to $12.5 billion.
However, operating expenses increased significantly due to R&D investments.
Key Metrics:
- Revenue: $12.5B (up 45% YoY)
- Net Income: $3.2B (up 12% YoY)
- EPS: $2.45
Risks & Challenges:
Supply chain constraints remain a bottleneck. Geopolitical tensions in Asia
could impact our semiconductor fabrication partners.
Additionally, a new lawsuit from a competitor regarding patent infringement
could result in fines up to $500M.
Outlook:
We expect Q4 revenue to be flat due to seasonal adjustments.




### 🧠 4. Analityk AI (Gemini 2.5 Flash)

Teraz najważniejsza część. Wyślemy ten tekst do modelu Google.
Używamy `ChatGoogleGenerativeAI` z modelem **gemini-2.5-flash**.

In [10]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Ładowanie klucza
load_dotenv(os.path.join(project_root, ".env"))

if not os.getenv("GOOGLE_API_KEY"):
    print("❌ BŁĄD: Brak klucza Google API w .env")
else:
    print("✅ Klucz Google załadowany.")

# Konfiguracja modelu
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0  # Zero kreatywności, chcemy fakty
)

✅ Klucz Google załadowany.


In [11]:
def analyze_report(report_text):
    """Wysyła tekst raportu do AI i zwraca analizę."""
    
    # Prompt Systemowy (Rola)
    system_prompt = """
    Jesteś Starszym Analitykiem Inwestycyjnym (Senior CFA). 
    Twoim zadaniem jest analiza tekstu raportu finansowego.
    
    Przygotuj zwięzłe podsumowanie w języku POLSKIM zawierające:
    1. 🟢 Główne sukcesy (Revenue, Growth).
    2. 🔴 Główne ryzyka (Risks, Supply Chain, Lawsuits).
    3. ⚖️ Werdykt inwestycyjny (Jedno zdanie: Bullish/Bearish/Neutral).
    """
    
    # Prompt Użytkownika (Dane)
    user_prompt = f"Oto treść raportu:\n\n{report_text}"
    
    print("🤖 Analityk AI (Gemini) pracuje...")
    
    try:
        # LangChain obsługuje formatowanie listy wiadomości
        response = llm.invoke([
            ("system", system_prompt),
            ("human", user_prompt)
        ])
        return response.content
    except Exception as e:
        return f"Błąd API: {e}"

# Uruchomienie
analysis = analyze_report(raw_text)

print("\n" + "="*50)
print("📊 RAPORT ANALITYCZNY AI")
print("="*50)
print(analysis)

🤖 Analityk AI (Gemini) pracuje...

📊 RAPORT ANALITYCZNY AI
Oto zwięzłe podsumowanie:

1.  **🟢 Główne sukcesy:**
    FutureCorp odnotowało silne wyniki, napędzane bezprecedensowym popytem na układy AI. Przychody wzrosły o 45% r/r do 12,5 mld USD, a dochód netto o 12% r/r do 3,2 mld USD.

2.  **🔴 Główne ryzyka:**
    Firma stoi w obliczu ograniczeń łańcucha dostaw oraz ryzyka związanego z napięciami geopolitycznymi w Azji, które mogą wpłynąć na partnerów produkcyjnych. Dodatkowo, nowy pozew o naruszenie patentów może skutkować grzywną do 500 mln USD, a koszty operacyjne znacząco wzrosły z powodu inwestycji w R&D.

3.  **⚖️ Werdykt inwestycyjny:**
    Neutralny.


## 🧠 Zadanie Domowe: Dynamiczny Analityk

Obecnie nasz bot robi ogólne podsumowanie.
Zmodyfikuj go, aby przyjmował parametr `mode`.

**Zadanie:**
1. Napisz funkcję `analyze_custom(text, mode)`.
2. Jeśli `mode == "RISK"`, prompt ma brzmieć: *"Jesteś Risk Managerem. Wypisz tylko listę zagrożeń w punktach."*
3. Jeśli `mode == "GROWTH"`, prompt ma brzmieć: *"Jesteś Inwestorem Wzrostowym. Wypisz tylko szanse i wzrosty."*
4. Przetestuj oba tryby na naszym raporcie.

In [12]:
def analyze_custom(text, mode="GENERAL"):
    
    if mode == "RISK":
        role = "Jesteś Risk Managerem. Wypisz w punktach WSZYSTKIE zagrożenia (prawne, łańcuch dostaw, finansowe). Bądź krytyczny."
    elif mode == "GROWTH":
        role = "Jesteś Inwestorem Growth. Skup się tylko na wzroście przychodów, nowych produktach i ekspansji. Ignoruj ryzyka."
    else:
        role = "Jesteś analitykiem finansowym. Zrób ogólne podsumowanie."
        
    print(f"\n🕵️‍♂️ Tryb analizy: {mode}")
    
    response = llm.invoke([
        ("system", role),
        ("human", f"Raport:\n{text}")
    ])
    
    return response.content

# Testy
print(analyze_custom(raw_text, mode="RISK"))
print("-" * 40)
print(analyze_custom(raw_text, mode="GROWTH"))


🕵️‍♂️ Tryb analizy: RISK
Rozumiem. Jako Risk Manager, analizuję dostarczony raport FutureCorp Inc. z krytycznym spojrzeniem, identyfikując wszystkie potencjalne zagrożenia prawne, związane z łańcuchem dostaw oraz finansowe.

---

**Raport Ocena Ryzyka dla FutureCorp Inc. - Q3 2024**

**I. Ryzyka Prawne:**

*   **Bieżący Pozew o Naruszenie Patentów:** Bezpośrednie ryzyko ze strony konkurenta, z potencjalną karą finansową do **$500M**. To nie tylko koszt kary, ale także:
    *   Koszty sądowe i obsługi prawnej.
    *   Rozproszenie uwagi zarządu i kluczowych pracowników.
    *   Potencjalne negatywne skutki dla reputacji firmy, niezależnie od wyniku procesu.
    *   Możliwość nałożenia zakazu sprzedaży produktów lub konieczności zmiany technologii, co wpłynie na przyszłe przychody i koszty R&D.
*   **Ryzyko Przyszłych Sporów Patentowych:** Działanie w szybko rozwijającej się branży technologicznej (AI Hardware) oraz znaczące inwestycje w R&D zwiększają prawdopodobieństwo przyszłych spor